In [8]:
import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture("3150-166336014_tiny.mp4")  # Change to your video file

# Read the first frame and convert it to grayscale
ret, old_frame = cap.read()
if not ret:
    print("Error: Cannot read video file")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Detect good features to track using Shi-Tomasi Corner Detection
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Parameters for Lucas-Kanade Optical Flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create a mask image for drawing motion tracks
mask = np.zeros_like(old_frame)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute optical flow (track points)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None and p0 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw motion tracks
        for new, old in zip(good_new, good_old):
            a, b = map(int, new.ravel())
            c, d = map(int, old.ravel())
            cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
            cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

        img = cv2.add(frame, mask)

        # Show the result
        cv2.imshow('Optical Flow - Lucas-Kanade', img)

        # Update previous frame and points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # Exit when 'Esc' is pressed
    if cv2.waitKey(30) & 0xFF == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load two consecutive frames
frame1 = cv2.imread("frame1.png")
frame2 = cv2.imread("frame2.png")

# Convert frames to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

# Detect good features to track in the first frame
p0 = cv2.goodFeaturesToTrack(gray1, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Lucas-Kanade Optical Flow parameters
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Compute optical flow
p1, st, err = cv2.calcOpticalFlowPyrLK(gray1, gray2, p0, None, **lk_params)

# Select good points
good_new = p1[st == 1]
good_old = p0[st == 1]

# Draw the motion vectors
for i, (new, old) in enumerate(zip(good_new, good_old)):
    a, b = new.ravel()
    c, d = old.ravel()
    frame2 = cv2.line(frame2, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
    frame2 = cv2.circle(frame2, (int(a), int(b)), 5, (0, 0, 255), -1)

cv2.imshow("Optical Flow", frame2)
cv2.waitKey(0)
cv2.destroyAllWindows()